In [16]:
import sys

sys.path.append('../')
from auton_survival import datasets
outcomes, features = datasets.load_support()

In [17]:
from auton_survival.preprocessing import Preprocessor
cat_feats = ['sex', 'dzgroup', 'dzclass', 'income', 'race', 'ca']
num_feats = ['age', 'num.co', 'meanbp', 'wblc', 'hrt', 'resp', 
	     'temp', 'pafi', 'alb', 'bili', 'crea', 'sod', 'ph', 
             'glucose', 'bun', 'urine', 'adlp', 'adls']

features = Preprocessor().fit_transform(features, cat_feats=cat_feats, num_feats=num_feats)

In [11]:
import numpy as np
horizons = [0.25, 0.5, 0.75, 0.9]
times = np.quantile(outcomes.time[outcomes.event==1], horizons).tolist()

NameError: name 'outcomes' is not defined

In [21]:
x, t, e = features.values, outcomes.time.values, outcomes.event.values

n = len(x)

tr_size = int(n*0.70)
vl_size = int(n*0.10)
te_size = int(n*0.20)

x_train, x_test, x_val = x[:tr_size], x[-te_size:], x[tr_size:tr_size+vl_size]
t_train, t_test, t_val = t[:tr_size], t[-te_size:], t[tr_size:tr_size+vl_size]
e_train, e_test, e_val = e[:tr_size], e[-te_size:], e[tr_size:tr_size+vl_size]

In [14]:
from pycox.datasets import from_kkbox
# test

kkbox_data = from_kkbox._DatasetKKBoxChurn()
#kkbox_data.download_kkbox()

df = kkbox_data.read_df()

import numpy as np
import pandas as pd

e = np.array(df.event)
t = np.array(df.duration)
x = df.drop(columns=['event','duration','msno'])

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
x['gender'] = le.fit_transform(x['gender'])
x['registered_via'] = le.fit_transform(x['registered_via'])
x['city'] = le.fit_transform(x['city'])
x = np.array(x).astype(float)

import os, sys
import numpy as np 

# path = '/home/r10user10/Documents/Jiacheng/dspm-auton-survival'
# os.chdir(path)
# print(os.getcwd())

from auton_survival import datasets

n = len(x)

tr_size = int(n * 0.70)
vl_size = int(n * 0.10)
te_size = int(n * 0.20)

print(tr_size, vl_size, te_size)

x_train, x_test, x_val = x[:tr_size], x[-te_size:], x[tr_size:tr_size + vl_size]
t_train, t_test, t_val = t[:tr_size], t[-te_size:], t[tr_size:tr_size + vl_size]
e_train, e_test, e_val = e[:tr_size], e[-te_size:], e[tr_size:tr_size + vl_size]

1250450 178635 357271


In [24]:
from auton_survival import DeepCoxPH

model = DeepCoxPH(layers=[100,100])
model.fit(x_train, t_train, e_train, iters=100, learning_rate=1e-4)


  0%|          | 0/100 [00:00<?, ?it/s]

 36%|███▌      | 36/100 [00:03<00:06,  9.68it/s]


In [25]:
horizons = [0.25, 0.5, 0.75, 0.9]
times = np.quantile(t[e == 1], horizons).tolist()
print(times)
out_risk = model.predict_risk(x_test, times)
print(out_risk[0])
out_survival = model.predict_survival(x_test, times)
print(out_survival[0])
from sksurv.metrics import concordance_index_ipcw, brier_score, cumulative_dynamic_auc

cis = []
brs = []

et_train = np.array([(e_train[i], t_train[i]) for i in range(len(e_train))],
                 dtype = [('e', bool), ('t', float)])
#print(et_train)
et_test = np.array([(e_test[i], t_test[i]) for i in range(len(e_test))],
                 dtype = [('e', bool), ('t', float)])
# et_val = np.array([(e_val[i], t_val[i]) for i in range(len(e_val))],
#                  dtype = [('e', bool), ('t', float)])
# print(et_train[0:10])
for i, _ in enumerate(times):
    cis.append(concordance_index_ipcw(et_train, et_test, out_risk[:, i], times[i])[0])
brs.append(brier_score(et_train, et_test, out_survival, times)[1])
roc_auc = []
for i, _ in enumerate(times):
    roc_auc.append(cumulative_dynamic_auc(et_train, et_test, out_risk[:, i], times[i])[0])
for horizon in enumerate(horizons):
    print(f"For {horizon[1]} quantile")
    print("TD Concordance Index:", cis[horizon[0]])
    print("Brier Score:", brs[0][horizon[0]])
    print("ROC AUC ", roc_auc[horizon[0]][0], "\n")

[14.0, 58.0, 252.0, 634.0]
[0.14317213 0.30113581 0.48111739 0.6118528 ]
[0.85682787 0.69886419 0.51888261 0.3881472 ]
For 0.25 quantile
TD Concordance Index: 0.6871456067024558
Brier Score: 0.11852332162339607
ROC AUC  0.6929623119145665 

For 0.5 quantile
TD Concordance Index: 0.6668667540970674
Brier Score: 0.1945653944322709
ROC AUC  0.6911809510417398 

For 0.75 quantile
TD Concordance Index: 0.6662968752911048
Brier Score: 0.22173971610095355
ROC AUC  0.7140522202701884 

For 0.9 quantile
TD Concordance Index: 0.665253720853873
Brier Score: 0.1780258488156854
ROC AUC  0.7138424641970121 



In [26]:
# from pycox.datasets import from_kkbox
# # test

# kkbox_data = from_kkbox._DatasetKKBoxChurn()
# #kkbox_data.download_kkbox()

# df = kkbox_data.read_df()

# import numpy as np
# import pandas as pd

# e = np.array(df.event)
# t = np.array(df.duration)
# x = df.drop(columns=['event','duration','msno'])

# from sklearn.preprocessing import LabelEncoder

# le = LabelEncoder()
# x['gender'] = le.fit_transform(x['gender'])
# x['registered_via'] = le.fit_transform(x['registered_via'])
# x['city'] = le.fit_transform(x['city'])
# x = np.array(x).astype(float)

# import os, sys
# import numpy as np 

# # path = '/home/r10user10/Documents/Jiacheng/dspm-auton-survival'
# # os.chdir(path)
# # print(os.getcwd())

# from auton_survival import datasets

# n = len(x)

# tr_size = int(n * 0.70)
# vl_size = int(n * 0.10)
# te_size = int(n * 0.20)

# print(tr_size, vl_size, te_size)

# x_train, x_test, x_val = x[:tr_size], x[-te_size:], x[tr_size:tr_size + vl_size]
# t_train, t_test, t_val = t[:tr_size], t[-te_size:], t[tr_size:tr_size + vl_size]
# e_train, e_test, e_val = e[:tr_size], e[-te_size:], e[tr_size:tr_size + vl_size]

1250450 178635 357271


In [27]:
from sksurv.linear_model import CoxPHSurvivalAnalysis

cph = CoxPHSurvivalAnalysis()
cph.fit(x_train, et_train)
pre = cph.predict(x_test)

et_train = np.array([(e_train[i], t_train[i]) for i in range(len(e_train))],
                 dtype = [('e', bool), ('t', float)])
et_test = np.array([(e_test[i], t_test[i]) for i in range(len(e_test))],
                 dtype = [('e', bool), ('t', float)])

concordance_index_ipcw(et_train, et_test, pre)[0]

ValueError: Found input variables with inconsistent numbers of samples: [1250450, 6373, 6373]

In [ ]:
import sys

sys.path.append('../')
from auton_survival import datasets
outcomes, features = datasets.load_support()
from auton_survival.preprocessing import Preprocessor
cat_feats = ['sex', 'dzgroup', 'dzclass', 'income', 'race', 'ca']
num_feats = ['age', 'num.co', 'meanbp', 'wblc', 'hrt', 'resp', 
	     'temp', 'pafi', 'alb', 'bili', 'crea', 'sod', 'ph', 
             'glucose', 'bun', 'urine', 'adlp', 'adls']

features = Preprocessor().fit_transform(features, cat_feats=cat_feats, num_feats=num_feats)

/home/r10user10/Documents/anaconda3/envs/python310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import numpy as np
horizons = [0.25, 0.5, 0.75, 0.9]
times = np.quantile(outcomes.time[outcomes.event==1], horizons).tolist()

In [ ]:
x, t, e = features.values, outcomes.time.values, outcomes.event.values

n = len(x)

tr_size = int(n*0.70)
vl_size = int(n*0.10)
te_size = int(n*0.20)

x_train, x_test, x_val = x[:tr_size], x[-te_size:], x[tr_size:tr_size+vl_size]
t_train, t_test, t_val = t[:tr_size], t[-te_size:], t[tr_size:tr_size+vl_size]
e_train, e_test, e_val = e[:tr_size], e[-te_size:], e[tr_size:tr_size+vl_size]

In [ ]:
from auton_survival.models.dsm import DeepSurvivalMachines
model = DeepSurvivalMachines(k = 3,
                                distribution = 'LogNormal',
                                layers = [100,100])
# The fit method is called to train the model
model.fit(x_train, t_train, e_train, iters = 100, learning_rate = 1e-5)

100%|██████████| 100/100 [00:19<00:00,  5.23it/s]


In [ ]:
out_risk = model.predict_risk(x_test, times)
out_survival = model.predict_survival(x_test, times)

from sksurv.metrics import concordance_index_ipcw, brier_score, cumulative_dynamic_auc

cis = []
brs = []

et_train = np.array([(e_train[i], t_train[i]) for i in range(len(e_train))],
                 dtype = [('e', bool), ('t', float)])
#print(et_train)
et_test = np.array([(e_test[i], t_test[i]) for i in range(len(e_test))],
                 dtype = [('e', bool), ('t', float)])
# et_val = np.array([(e_val[i], t_val[i]) for i in range(len(e_val))],
#                  dtype = [('e', bool), ('t', float)])
# print(et_train[0:10])
for i, _ in enumerate(times):
    cis.append(concordance_index_ipcw(et_train, et_test, out_risk[:, i], times[i])[0])
brs.append(brier_score(et_train, et_test, out_survival, times)[1])
roc_auc = []
for i, _ in enumerate(times):
    roc_auc.append(cumulative_dynamic_auc(et_train, et_test, out_risk[:, i], times[i])[0])
for horizon in enumerate(horizons):
    print(f"For {horizon[1]} quantile")
    print("TD Concordance Index:", cis[horizon[0]])
    print("Brier Score:", brs[0][horizon[0]])
    print("ROC AUC ", roc_auc[horizon[0]][0], "\n")

For 0.25 quantile
TD Concordance Index: 0.7232312177449028
Brier Score: 0.11617837587122909
ROC AUC  0.7303541612601541 

For 0.5 quantile
TD Concordance Index: 0.6772881456209202
Brier Score: 0.19385375149447842
ROC AUC  0.6920261196653308 

For 0.75 quantile
TD Concordance Index: 0.642599458354434
Brier Score: 0.23662224079581234
ROC AUC  0.6883201066214334 

For 0.9 quantile
TD Concordance Index: 0.6177963553329717
Brier Score: 0.18929803538852297
ROC AUC  0.6945794222304769 



In [1]:
def support():
    import sys
    sys.path.append('../')
    from auton_survival import datasets
    outcomes, features = datasets.load_support()
    from auton_survival.preprocessing import Preprocessor
    cat_feats = ['sex', 'dzgroup', 'dzclass', 'income', 'race', 'ca']
    num_feats = ['age', 'num.co', 'meanbp', 'wblc', 'hrt', 'resp', 
            'temp', 'pafi', 'alb', 'bili', 'crea', 'sod', 'ph', 
                'glucose', 'bun', 'urine', 'adlp', 'adls']

    features = Preprocessor().fit_transform(features, cat_feats=cat_feats, num_feats=num_feats)
    x, t, e = features.values, outcomes.time.values, outcomes.event.values

    n = len(x)

    tr_size = int(n*0.80)
    te_size = int(n*0.20)

    x_train, x_test = x[:tr_size], x[-te_size:]
    t_train, t_test = t[:tr_size], t[-te_size:]
    e_train, e_test = e[:tr_size], e[-te_size:]
    return x_train, t_train , e_train, x_test, t_test , e_test

In [2]:
def synthetic():
    import pandas as pd
    import torch
    from tqdm import tqdm 
    import sys
    sys.path.append('../')

    from auton_survival.datasets import load_dataset

    # Load the synthetic dataset
    outcomes, features, interventions = load_dataset(dataset='SYNTHETIC')

    # Hyper-parameters
    random_seed = 0
    test_size = 0.25

    # Split the synthetic data into training and testing data
    import numpy as np

    np.random.seed(random_seed)
    n = features.shape[0] 

    test_idx = np.zeros(n).astype('bool')
    test_idx[np.random.randint(n, size=int(n*test_size))] = True 

    features_tr = features.iloc[~test_idx] 
    outcomes_tr = outcomes.iloc[~test_idx]
    interventions_tr = interventions[~test_idx]
    print(f'Number of training data points: {len(features_tr)}')

    features_te = features.iloc[test_idx] 
    outcomes_te = outcomes.iloc[test_idx]
    interventions_te = interventions[test_idx]
    print(f'Number of test data points: {len(features_te)}')

    interventions_tr.name, interventions_te.name = 'treat', 'treat'
    features_tr_dcph = pd.concat([features_tr, interventions_tr.astype('float64')], axis=1)
    features_te_dcph = pd.concat([features_te, interventions_te.astype('float64')], axis=1)
    outcomes_tr_dcph = pd.DataFrame(outcomes_tr, columns=['event', 'time']).astype('float64')


    x_train = features_tr_dcph.values
    e_train = outcomes_tr['event'].values.astype(float)
    t_train = outcomes_tr['time'].values

    x_test = features_te_dcph.values
    e_test = outcomes_te['event'].values.astype(float)
    t_test = outcomes_te['time'].values
    return x_train, t_train , e_train, x_test, t_test , e_test


In [3]:
def kkbox():
    from pycox.datasets import from_kkbox


    kkbox_data = from_kkbox._DatasetKKBoxChurn()
    #kkbox_data.download_kkbox()

    df = kkbox_data.read_df()

    import numpy as np
    import pandas as pd

    e = np.array(df.event)
    t = np.array(df.duration)
    x = df.drop(columns=['event','duration','msno'])

    from sklearn.preprocessing import LabelEncoder

    le = LabelEncoder()
    x['gender'] = le.fit_transform(x['gender'])
    x['registered_via'] = le.fit_transform(x['registered_via'])
    x['city'] = le.fit_transform(x['city'])
    x = np.array(x).astype(float)

    import os, sys
    import numpy as np 

    # path = '/home/r10user10/Documents/Jiacheng/dspm-auton-survival'
    # os.chdir(path)
    # print(os.getcwd())

    from auton_survival import datasets

    n = len(x)

    tr_size = int(n * 0.80)
    te_size = int(n * 0.20)


    x_train, x_test = x[:tr_size], x[-te_size:]
    t_train, t_test = t[:tr_size], t[-te_size:]
    e_train, e_test = e[:tr_size], e[-te_size:]
    return x_train, t_train , e_train, x_test, t_test , e_test

In [4]:
def mimic():
    import numpy as np
    x_train = np.load('x_train.npy')
    t_train = np.load('t_train.npy')
    e_train = 1 - np.load('e_train.npy')
    index = np.where(t_train <= 0)[0]
    t_train = np.delete(t_train, index)
    e_train = np.delete(e_train, index)
    x_train = np.delete(x_train, index, axis=0)
    x_test = np.load('x_test.npy')
    t_test = np.load('t_test.npy')
    e_test = 1 - np.load('e_test.npy')
    index = np.where(t_test <= 0)[0]
    t_test = np.delete(t_test, index)
    e_test = np.delete(e_test, index)
    x_test = np.delete(x_test, index, axis=0)
    x_train = np.mean(x_train, axis=1)
    x_test = np.mean(x_test, axis=1)
    print(x_train.shape)
    return x_train, t_train , e_train, x_test, t_test , e_test

In [17]:
def baseline(baseline, dataset, lr):
    if baseline == 'DeepCox':
        from auton_survival import DeepCoxPH
        model = DeepCoxPH(layers=[100,100])
    if baseline == 'DSM':
        from auton_survival.models.dsm import DeepSurvivalMachines
        model = DeepSurvivalMachines(k = 3,
                                distribution = 'LogNormal',
                                layers = [100,100])
    if baseline == 'DCM':
        from auton_survival.models.dcm import DeepCoxMixtures
        model = DeepCoxMixtures(k = 3, layers = [100,100])

    if dataset == 'support':
        x_train, t_train , e_train, x_test, t_test , e_test = support()
    if dataset == 'synthetic':
        x_train, t_train , e_train, x_test, t_test , e_test = synthetic()
    if dataset == 'kkbox':
        x_train, t_train , e_train, x_test, t_test , e_test = kkbox()
    if dataset == 'mimic':
        x_train, t_train , e_train, x_test, t_test , e_test = mimic()   
    
    model.fit(x_train, t_train, e_train, iters = 100, learning_rate = lr)
    horizons = [0.25, 0.5, 0.75, 0.9]
    x = np.concatenate((x_train, x_test), axis=0)
    t = np.concatenate((t_train, t_test), axis=0)
    e = np.concatenate((e_train, e_test), axis=0)
    times = np.quantile(t[e==1], horizons).tolist()
    out_risk = 1 - model.predict_survival(x_test, times)
    out_survival = model.predict_survival(x_test, times)

    from sksurv.metrics import concordance_index_ipcw, brier_score, cumulative_dynamic_auc

    cis = []
    brs = []

    et_train = np.array([(e_train[i], t_train[i]) for i in range(len(e_train))],
                    dtype = [('e', bool), ('t', float)])
    #print(et_train)
    et_test = np.array([(e_test[i], t_test[i]) for i in range(len(e_test))],
                    dtype = [('e', bool), ('t', float)])
    # et_val = np.array([(e_val[i], t_val[i]) for i in range(len(e_val))],
    #                  dtype = [('e', bool), ('t', float)])
    # print(et_train[0:10])
    for i, _ in enumerate(times):
        cis.append(concordance_index_ipcw(et_train, et_test, out_risk[:, i], times[i])[0])
    brs.append(brier_score(et_train, et_test, out_survival, times)[1])
    roc_auc = []
    for i, _ in enumerate(times):
        roc_auc.append(cumulative_dynamic_auc(et_train, et_test, out_risk[:, i], times[i])[0])
    for horizon in enumerate(horizons):
        print(f"For {horizon[1]} quantile")
        print("TD Concordance Index:", cis[horizon[0]])
        print("Brier Score:", brs[0][horizon[0]])
        print("ROC AUC ", roc_auc[horizon[0]][0], "\n")
    

In [18]:
import numpy as np

In [23]:

baseline('DCM', 'support', 1e-2)

 14%|█▍        | 14/100 [00:13<01:20,  1.07it/s]


For 0.25 quantile
TD Concordance Index: 0.7559910747138908
Brier Score: 0.1157773954978373
ROC AUC  0.7627809958429018 

For 0.5 quantile
TD Concordance Index: 0.6944677263494192
Brier Score: 0.1948717292185306
ROC AUC  0.7156219972345262 

For 0.75 quantile
TD Concordance Index: 0.6549660158903023
Brier Score: 0.23636025976841915
ROC AUC  0.6878754467801539 

For 0.9 quantile
TD Concordance Index: 0.6434117883039532
Brier Score: 0.18834185588483432
ROC AUC  0.6882486352760012 



In [24]:
baseline('DCM', 'synthetic', 1e-3)

Number of training data points: 3899
Number of test data points: 1101


  0%|          | 0/100 [00:00<?, ?it/s]

/home/r10user10/Documents/Jiacheng/dspm-auton-survival/auton_survival/models/dcm/dcm_utilities.py:58: RuntimeWarning: invalid value encountered in power
  return spl(ts)**risks
/home/r10user10/Documents/Jiacheng/dspm-auton-survival/auton_survival/models/dcm/dcm_utilities.py:53: RuntimeWarning: invalid value encountered in power
  s0ts = (-risks)*(spl(ts)**(risks-1))
  9%|▉         | 9/100 [00:06<01:05,  1.38it/s]

For 0.25 quantile
TD Concordance Index: 0.6750917306531483
Brier Score: 0.15080291947248434
ROC AUC  0.7029677810353033 

For 0.5 quantile
TD Concordance Index: 0.6783322871962039
Brier Score: 0.20979851676343117
ROC AUC  0.7320758415059773 

For 0.75 quantile
TD Concordance Index: 0.6803062204575463
Brier Score: 0.15477046739848743
ROC AUC  0.8128868303361304 

For 0.9 quantile
TD Concordance Index: 0.6752246957509107
Brier Score: 0.09295212037556035
ROC AUC  0.8197751184458351 



In [25]:
baseline('DCM', 'kkbox', 1e-2)

  0%|          | 0/100 [00:00<?, ?it/s]/home/r10user10/Documents/Jiacheng/dspm-auton-survival/auton_survival/models/dcm/dcm_utilities.py:58: RuntimeWarning: invalid value encountered in power
  return spl(ts)**risks
/home/r10user10/Documents/Jiacheng/dspm-auton-survival/auton_survival/models/dcm/dcm_utilities.py:53: RuntimeWarning: invalid value encountered in power
  s0ts = (-risks)*(spl(ts)**(risks-1))
/home/r10user10/Documents/anaconda3/envs/python310/lib/python3.10/site-packages/scipy/interpolate/_fitpack2.py:313: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)
/home/r10user10/Documents/Jiacheng/dspm-auton-survival/auton_survival/models/dcm/dcm_utilities.py:58: RuntimeWarning: overflow encountered in power
  

Exception!!!: probability tensor contains either `inf`, `nan` or element < 0


Exception!!!: probability tensor contains either `inf`, `nan` or element < 0


/home/r10user10/Documents/Jiacheng/dspm-auton-survival/auton_survival/models/dcm/dcm_utilities.py:53: RuntimeWarning: overflow encountered in multiply
  s0ts = (-risks)*(spl(ts)**(risks-1))


Exception!!!: probability tensor contains either `inf`, `nan` or element < 0


...

Exception!!!: probability tensor contains either `inf`, `nan` or element < 0


Exception!!!: probability tensor contains either `inf`, `nan` or element < 0


  3%|▎         | 3/100 [3:31:59<114:14:32, 4239.92s/it]

Exception!!!: probability tensor contains either `inf`, `nan` or element < 0


RuntimeError: probability tensor contains either `inf`, `nan` or element < 0